<a href="https://colab.research.google.com/github/john-d-noble/callcenter/blob/main/Call_Center_Forecasting_V1_Models_Complete_Implementation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# %% [markdown]
# # Call Center Forecasting V1 Models - Complete Implementation

# %% Hardware Check (CRITICAL: Must be first to define ENABLE_NEURAL)
print("🖥️ COMPUTATIONAL ENVIRONMENT CHECK")
print("=" * 50)

# GPU Check
try:
    gpu_info = !nvidia-smi
    gpu_info = '\n'.join(gpu_info)
    if gpu_info.find('failed') >= 0:
        print('❌ Not connected to a GPU')
        print('💡 Neural models (LSTM, CNN) will run on CPU (slower)')
        GPU_AVAILABLE = False
    else:
        print('✅ GPU Available:')
        print(gpu_info)
        GPU_AVAILABLE = True
except:
    print('❌ GPU check failed - assuming no GPU')
    GPU_AVAILABLE = False

# RAM Check
import psutil

ram_gb = psutil.virtual_memory().total / 1e9
print(f'\n💾 RAM Status: {ram_gb:.1f} GB available')

if ram_gb < 20:
    print('⚠️ Standard RAM - may limit large ensemble grid searches')
    HIGH_RAM = False
else:
    print('✅ High-RAM runtime - can handle complex model combinations!')
    HIGH_RAM = True

# Set computational strategy based on resources
print(f"\n🎯 COMPUTATIONAL STRATEGY:")
if GPU_AVAILABLE and HIGH_RAM:
    print("   🚀 FULL POWER: GPU + High RAM - All models enabled")
    ENABLE_NEURAL = True
    ENABLE_LARGE_GRIDS = True
elif GPU_AVAILABLE:
    print("   ⚡ GPU enabled, moderate RAM - Neural models OK, smaller grids")
    ENABLE_NEURAL = True
    ENABLE_LARGE_GRIDS = False
elif HIGH_RAM:
    print("   🧠 High RAM, no GPU - Large ensembles OK, neural models slower")
    ENABLE_NEURAL = False  # Still possible but slower
    ENABLE_LARGE_GRIDS = True
else:
    print("   💡 Standard setup - Focus on efficient models")
    ENABLE_NEURAL = False
    ENABLE_LARGE_GRIDS = False

print("=" * 50)

# %% Imports and Setup
print("\n📚 IMPORTING LIBRARIES")
print("=" * 30)

# Core libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime, timedelta
import warnings
warnings.filterwarnings('ignore')

# Statistical and time series
from scipy import stats
from scipy.stats import jarque_bera, shapiro, mode
from statsmodels.tsa.seasonal import seasonal_decompose
from statsmodels.tsa.stattools import adfuller, acf, pacf
from statsmodels.tsa.arima.model import ARIMA
from statsmodels.tsa.statespace.sarimax import SARIMAX
from statsmodels.tsa.holtwinters import ExponentialSmoothing
from statsmodels.tsa.exponential_smoothing.ets import ETSModel
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf
from statsmodels.stats.diagnostic import acorr_ljungbox

# Advanced time series models
try:
    from statsmodels.tsa.statespace.tools import diff
    from statsmodels.tsa.seasonal import STL
    ADVANCED_TS_AVAILABLE = True
    print("✅ Advanced time series models available")
except ImportError:
    ADVANCED_TS_AVAILABLE = False
    print("⚠️ Some advanced TS models may not be available")

# Prophet
try:
    from prophet import Prophet
    PROPHET_AVAILABLE = True
    print("✅ Prophet available")
except ImportError:
    PROPHET_AVAILABLE = False
    print("⚠️ Prophet not available")

# Machine Learning models
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.linear_model import LinearRegression, Ridge, Lasso, ElasticNet
from sklearn.svm import SVR
from sklearn.neural_network import MLPRegressor
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score, mean_absolute_percentage_error
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.model_selection import GridSearchCV, TimeSeriesSplit

# Neural networks (if GPU available)
if ENABLE_NEURAL:
    try:
        import tensorflow as tf
        from tensorflow.keras.models import Sequential
        from tensorflow.keras.layers import LSTM, Dense, Dropout, Conv1D, Flatten
        print("✅ TensorFlow/Keras available for neural models")
        KERAS_AVAILABLE = True
    except ImportError:
        print("⚠️ TensorFlow not available - skipping neural models")
        KERAS_AVAILABLE = False
        ENABLE_NEURAL = False
else:
    KERAS_AVAILABLE = False

# Visualization setup
plt.style.use('seaborn-v0_8')
sns.set_palette("husl")
plt.rcParams['figure.figsize'] = (12, 6)

# Model versioning
MODEL_VERSION = "V1"
print(f"\n🏷️ MODEL VERSION: {MODEL_VERSION}")
print("📊 Phase 1: Basic Statistical + Advanced Time Series + Hybrid Models")

print("\n✅ Setup Complete - Ready for Phase 1 Model Development!")

# %% Data Loading Function
def load_call_center_data_v1(file_path='enhanced_eda_data.csv'):
    """
    Load call center data with market integration for V1 models
    Includes data cleaning (trim first/last rows) and market feature creation
    """

    print("📁 LOADING CALL CENTER DATA (V1)")
    print("=" * 40)

    try:
        # Load main data file
        df = pd.read_csv(file_path, index_col='Date', parse_dates=True)
        print(f"✅ Loaded {len(df)} records from {file_path}")

        # Auto-detect call volume column
        volume_cols = ['calls', 'Calls', 'call_volume', 'Call_Volume', 'volume', 'Volume']
        volume_col = None

        for col in volume_cols:
            if col in df.columns:
                volume_col = col
                break

        if volume_col is None:
            numeric_cols = df.select_dtypes(include=[np.number]).columns
            volume_col = numeric_cols[0] if len(numeric_cols) > 0 else df.columns[0]

        print(f"🎯 Call volume column: {volume_col}")

        # Standardize column name
        if volume_col != 'calls':
            df = df.rename(columns={volume_col: 'calls'})

        # DATA CLEANING: Remove first and last rows (as requested)
        print("🧹 DATA CLEANING: Removing first and last rows")
        original_len = len(df)
        if len(df) > 2:
            first_date = df.index[0].strftime('%Y-%m-%d')
            last_date = df.index[-1].strftime('%Y-%m-%d')
            first_calls = df['calls'].iloc[0]
            last_calls = df['calls'].iloc[-1]

            print(f"   🗑️ Removing first: {first_date} ({first_calls:.0f} calls)")
            print(f"   🗑️ Removing last:  {last_date} ({last_calls:.0f} calls)")

            df = df.iloc[1:-1]
            print(f"   ✅ Cleaned: {original_len} → {len(df)} rows")

        # CHECK FOR MARKET DATA IN MAIN FILE
        print(f"\n📈 MARKET DATA INTEGRATION")
        print("-" * 25)

        expected_market_cols = [
            '^VIX_close', 'SPY_close', 'SPY_volume', 'QQQ_close', 'QQQ_volume',
            'DX-Y.NYB_close', 'GC=F_close', 'GC=F_volume', 'BTC-USD_close',
            'BTC-USD_volume', 'ETH-USD_close', 'ETH-USD_volume'
        ]

        existing_market_cols = [col for col in expected_market_cols if col in df.columns]

        if existing_market_cols:
            print(f"✅ Market data found: {len(existing_market_cols)} columns")
            for col in existing_market_cols[:5]:  # Show first 5
                print(f"   • {col}")
            if len(existing_market_cols) > 5:
                print(f"   • ... and {len(existing_market_cols)-5} more")

            # CREATE MARKET-DERIVED FEATURES
            print(f"\n🔧 Creating market-derived features...")

            # VIX features
            if '^VIX_close' in df.columns:
                df['vix_high'] = (df['^VIX_close'] > df['^VIX_close'].quantile(0.8)).astype(int)
                df['vix_spike'] = (df['^VIX_close'].pct_change() > 0.2).astype(int)
                print("   📈 VIX volatility features created")

            # Stock market features
            if 'SPY_close' in df.columns:
                df['spy_returns'] = df['SPY_close'].pct_change()
                df['market_stress'] = (df['spy_returns'] < -0.02).astype(int)
                df['spy_volatility'] = df['spy_returns'].rolling(7).std()
                print("   📉 Stock market stress features created")

            # Crypto features
            if 'BTC-USD_close' in df.columns:
                df['btc_returns'] = df['BTC-USD_close'].pct_change()
                df['crypto_volatility'] = df['btc_returns'].rolling(7).std()
                df['btc_extreme_move'] = (abs(df['btc_returns']) > 0.1).astype(int)
                print("   ₿ Crypto volatility features created")

            # Market uncertainty composite
            uncertainty_features = []
            if '^VIX_close' in df.columns:
                uncertainty_features.append(df['^VIX_close'])
            if 'spy_volatility' in df.columns:
                uncertainty_features.append(df['spy_volatility'] * 100)
            if 'crypto_volatility' in df.columns:
                uncertainty_features.append(df['crypto_volatility'] * 100)

            if uncertainty_features:
                uncertainty_matrix = pd.concat(uncertainty_features, axis=1)
                df['market_uncertainty_index'] = uncertainty_matrix.mean(axis=1)
                print("   🌊 Market uncertainty index created")

        else:
            print("⚠️ No market data columns found in main file")

        # Final data overview
        print(f"\n📊 FINAL DATASET OVERVIEW")
        print("-" * 25)
        print(f"   Date range: {df.index.min().strftime('%Y-%m-%d')} to {df.index.max().strftime('%Y-%m-%d')}")
        print(f"   Total days: {len(df)}")
        print(f"   Total columns: {len(df.columns)}")
        print(f"   Call volume range: {df['calls'].min():.0f} to {df['calls'].max():.0f}")
        print(f"   Missing values: {df.isnull().sum().sum()}")

        return df

    except Exception as e:
        print(f"❌ Error loading data: {e}")
        return None

# %% Load Data
df_raw = load_call_center_data_v1()

if df_raw is not None:
    print(f"\n✅ Data loading successful - Ready for cross-validation setup!")
else:
    print(f"\n❌ Data loading failed - Check file path and format")
    raise Exception("Data loading failed")

# %% Cross-Validation Setup
def create_time_series_splits_v1(df, n_splits=5, test_size=7, gap=0):
    """
    Create time series cross-validation splits BEFORE feature engineering
    This prevents data leakage by ensuring no future information in features
    """

    print("🔒 TIME SERIES CROSS-VALIDATION SETUP (V1)")
    print("=" * 45)
    print("⚠️ Creating splits BEFORE feature engineering to prevent data leakage")

    splits = []
    total_size = len(df)

    for i in range(n_splits):
        # Calculate split points (working backwards from end)
        test_end = total_size - i * test_size
        test_start = test_end - test_size
        train_end = test_start - gap

        if train_end < 30:  # Need minimum 30 days for training
            break

        train_idx = df.index[:train_end]
        test_idx = df.index[test_start:test_end]

        splits.append({
            'train_idx': train_idx,
            'test_idx': test_idx,
            'train_size': len(train_idx),
            'test_size': len(test_idx),
            'split_date': test_idx[0] if len(test_idx) > 0 else None
        })

    print(f"✅ Created {len(splits)} data-leakage-free splits:")
    for i, split in enumerate(splits):
        print(f"  Split {i+1}: Train {split['train_size']} days → Test {split['test_size']} days")
        print(f"    Train: {split['train_idx'][0].strftime('%Y-%m-%d')} to {split['train_idx'][-1].strftime('%Y-%m-%d')}")
        print(f"    Test:  {split['test_idx'][0].strftime('%Y-%m-%d')} to {split['test_idx'][-1].strftime('%Y-%m-%d')}")

    return splits

# Create CV splits
cv_splits = create_time_series_splits_v1(df_raw, n_splits=5, test_size=7, gap=0)

print(f"\n🔒 Cross-validation framework established")
print(f"📊 Ready for per-split feature engineering")

# %% Feature Engineering
def create_features_v1(df_train, df_test=None):
    """
    Create comprehensive feature set using ONLY training data statistics
    Apply same transformations to test data using training-derived parameters
    """

    print("🛠️ FEATURE ENGINEERING V1 (Leakage-Free)")
    print("=" * 40)
    print("✅ Using ONLY training data statistics")

    # Work on training data first
    df_features_train = df_train.copy()

    # TIME-BASED FEATURES (no leakage risk)
    df_features_train['year'] = df_features_train.index.year
    df_features_train['month'] = df_features_train.index.month
    df_features_train['day'] = df_features_train.index.day
    df_features_train['dayofweek'] = df_features_train.index.dayofweek
    df_features_train['dayofyear'] = df_features_train.index.dayofyear
    df_features_train['quarter'] = df_features_train.index.quarter
    df_features_train['week'] = df_features_train.index.isocalendar().week

    # CYCLICAL ENCODING (for better ML model performance)
    df_features_train['month_sin'] = np.sin(2 * np.pi * df_features_train['month'] / 12)
    df_features_train['month_cos'] = np.cos(2 * np.pi * df_features_train['month'] / 12)
    df_features_train['dow_sin'] = np.sin(2 * np.pi * df_features_train['dayofweek'] / 7)
    df_features_train['dow_cos'] = np.cos(2 * np.pi * df_features_train['dayofweek'] / 7)
    df_features_train['doy_sin'] = np.sin(2 * np.pi * df_features_train['dayofyear'] / 365.25)
    df_features_train['doy_cos'] = np.cos(2 * np.pi * df_features_train['dayofyear'] / 365.25)

    # BINARY FEATURES
    df_features_train['is_weekend'] = (df_features_train['dayofweek'] >= 5).astype(int)
    df_features_train['is_monday'] = (df_features_train['dayofweek'] == 0).astype(int)
    df_features_train['is_friday'] = (df_features_train['dayofweek'] == 4).astype(int)
    df_features_train['is_month_start'] = df_features_train.index.is_month_start.astype(int)
    df_features_train['is_month_end'] = df_features_train.index.is_month_end.astype(int)

    # LAG FEATURES (using only training data)
    for lag in [1, 2, 3, 7]:
        df_features_train[f'calls_lag_{lag}'] = df_features_train['calls'].shift(lag)

    # ROLLING STATISTICS (using only training data)
    for window in [7, 14, 30]:
        df_features_train[f'calls_mean_{window}d'] = df_features_train['calls'].rolling(window).mean()
        df_features_train[f'calls_std_{window}d'] = df_features_train['calls'].rolling(window).std()
        df_features_train[f'calls_min_{window}d'] = df_features_train['calls'].rolling(window).min()
        df_features_train[f'calls_max_{window}d'] = df_features_train['calls'].rolling(window).max()

    # MARKET FEATURES (if available, using training thresholds only)
    market_features_created = 0
    if '^VIX_close' in df_features_train.columns:
        train_vix_high_threshold = df_features_train['^VIX_close'].quantile(0.8)
        df_features_train['vix_high_train'] = (df_features_train['^VIX_close'] > train_vix_high_threshold).astype(int)
        market_features_created += 1

    if 'spy_returns' in df_features_train.columns:
        df_features_train['market_stress_train'] = (df_features_train['spy_returns'] < -0.02).astype(int)
        market_features_created += 1

    # Count features created
    total_features = len(df_features_train.columns) - len(df_train.columns)

    print(f"✅ Created {total_features} features for training data")

    # Apply same transformations to test data if provided
    if df_test is not None:
        df_features_test = df_test.copy()

        # Apply same time-based features
        df_features_test['year'] = df_features_test.index.year
        df_features_test['month'] = df_features_test.index.month
        df_features_test['day'] = df_features_test.index.day
        df_features_test['dayofweek'] = df_features_test.index.dayofweek
        df_features_test['dayofyear'] = df_features_test.index.dayofyear
        df_features_test['quarter'] = df_features_test.index.quarter
        df_features_test['week'] = df_features_test.index.isocalendar().week

        # Apply same cyclical encoding
        df_features_test['month_sin'] = np.sin(2 * np.pi * df_features_test['month'] / 12)
        df_features_test['month_cos'] = np.cos(2 * np.pi * df_features_test['month'] / 12)
        df_features_test['dow_sin'] = np.sin(2 * np.pi * df_features_test['dayofweek'] / 7)
        df_features_test['dow_cos'] = np.cos(2 * np.pi * df_features_test['dayofweek'] / 7)
        df_features_test['doy_sin'] = np.sin(2 * np.pi * df_features_test['dayofyear'] / 365.25)
        df_features_test['doy_cos'] = np.cos(2 * np.pi * df_features_test['dayofyear'] / 365.25)

        # Apply same binary features
        df_features_test['is_weekend'] = (df_features_test['dayofweek'] >= 5).astype(int)
        df_features_test['is_monday'] = (df_features_test['dayofweek'] == 0).astype(int)
        df_features_test['is_friday'] = (df_features_test['dayofweek'] == 4).astype(int)
        df_features_test['is_month_start'] = df_features_test.index.is_month_start.astype(int)
        df_features_test['is_month_end'] = df_features_test.index.is_month_end.astype(int)

        # For lag and rolling features, combine train+test data but respect temporal order
        combined_data = pd.concat([df_features_train['calls'], df_features_test['calls']])

        # Apply lag features
        for lag in [1, 2, 3, 7]:
            df_features_test[f'calls_lag_{lag}'] = combined_data.shift(lag).loc[df_features_test.index]

        # Apply rolling features
        for window in [7, 14, 30]:
            df_features_test[f'calls_mean_{window}d'] = combined_data.rolling(window).mean().loc[df_features_test.index]
            df_features_test[f'calls_std_{window}d'] = combined_data.rolling(window).std().loc[df_features_test.index]
            df_features_test[f'calls_min_{window}d'] = combined_data.rolling(window).min().loc[df_features_test.index]
            df_features_test[f'calls_max_{window}d'] = combined_data.rolling(window).max().loc[df_features_test.index]

        # Apply market features using TRAINING thresholds
        if '^VIX_close' in df_features_test.columns and '^VIX_close' in df_features_train.columns:
            df_features_test['vix_high_train'] = (df_features_test['^VIX_close'] > train_vix_high_threshold).astype(int)

        if 'spy_returns' in df_features_test.columns and 'spy_returns' in df_features_train.columns:
            df_features_test['market_stress_train'] = (df_features_test['spy_returns'] < -0.02).astype(int)

        print(f"✅ Applied same transformations to test data")
        return df_features_train, df_features_test

    return df_features_train, None

# %% Basic Statistical Models
class BasicStatisticalModels_V1:
    """Collection of basic statistical forecasting models - Version 1"""

    def __init__(self):
        self.models = {}
        self.model_version = "V1"

    def fit_mean_v1(self, y_train):
        """Mean V1: Simple historical average forecast"""
        self.models['mean'] = y_train.mean()
        return self

    def fit_median_v1(self, y_train):
        """Median V1: Robust central tendency (outlier resistant)"""
        self.models['median'] = y_train.median()
        return self

    def fit_naive_v1(self, y_train):
        """Naive V1: Last observed value"""
        self.models['naive'] = y_train.iloc[-1]
        return self

    def fit_seasonal_naive_v1(self, y_train, season_length=7):
        """Seasonal Naive V1: Last value from same season (BENCHMARK MODEL)"""
        if len(y_train) >= season_length:
            self.models['seasonal_naive'] = {
                'values': y_train.iloc[-season_length:],
                'season_length': season_length
            }
        else:
            self.models['seasonal_naive'] = {
                'values': y_train,
                'season_length': len(y_train)
            }
        return self

    def fit_drift_v1(self, y_train):
        """Drift V1: Linear trend from first to last observation"""
        n = len(y_train)
        if n > 1:
            slope = (y_train.iloc[-1] - y_train.iloc[0]) / (n - 1)
            self.models['drift'] = {
                'last_value': y_train.iloc[-1],
                'slope': slope
            }
        else:
            self.models['drift'] = {'last_value': y_train.iloc[-1], 'slope': 0}
        return self

    def predict(self, steps, model_type):
        """Generate forecasts for specified number of steps"""
        if model_type in ['mean', 'median', 'naive']:
            return np.full(steps, self.models[model_type])

        elif model_type == 'seasonal_naive':
            model_info = self.models['seasonal_naive']
            season_values = model_info['values'].values
            season_length = model_info['season_length']
            forecasts = []
            for i in range(steps):
                forecasts.append(season_values[-(season_length - (i % season_length))])
            return np.array(forecasts)

        elif model_type == 'drift':
            model_info = self.models['drift']
            last_value = model_info['last_value']
            slope = model_info['slope']
            return np.array([last_value + slope * (i + 1) for i in range(steps)])

def fit_all_basic_models_v1(y_train, forecast_steps):
    """Fit all basic statistical models and return predictions"""

    results = {}
    basic_models = BasicStatisticalModels_V1()

    # Fit all models
    basic_models.fit_mean_v1(y_train)
    basic_models.fit_median_v1(y_train)
    basic_models.fit_naive_v1(y_train)
    basic_models.fit_seasonal_naive_v1(y_train, season_length=7)
    basic_models.fit_drift_v1(y_train)

    # Generate predictions for all models
    model_names = ['mean', 'median', 'naive', 'seasonal_naive', 'drift']

    for model_name in model_names:
        try:
            pred = basic_models.predict(forecast_steps, model_name)
            results[f"{model_name}_{MODEL_VERSION}"] = pred
        except Exception as e:
            print(f"⚠️ {model_name} failed: {e}")
            results[f"{model_name}_{MODEL_VERSION}"] = np.full(forecast_steps, y_train.mean())

    return results

# %% Advanced Time Series Models
def fit_advanced_time_series_v1(y_train, forecast_steps):
    """Advanced time series models for more sophisticated pattern capture"""

    print("📈 FITTING ADVANCED TIME SERIES MODELS V1")

    results = {}

    # 1. ETS (Error, Trend, Seasonal)
    try:
        if len(y_train) >= 14:
            ets_model = ETSModel(
                y_train,
                error='add',
                trend='add',
                seasonal='add',
                seasonal_periods=7
            ).fit()
            ets_forecast = ets_model.forecast(steps=forecast_steps)
            results[f'ets_{MODEL_VERSION}'] = ets_forecast
        else:
            results[f'ets_{MODEL_VERSION}'] = np.full(forecast_steps, y_train.mean())
    except Exception as e:
        results[f'ets_{MODEL_VERSION}'] = np.full(forecast_steps, y_train.mean())

    # 2. Holt-Winters
    try:
        if len(y_train) >= 14:
            hw_model = ExponentialSmoothing(
                y_train,
                seasonal='add',
                seasonal_periods=7,
                trend='add'
            ).fit()
            hw_forecast = hw_model.forecast(steps=forecast_steps)
            results[f'holt_winters_{MODEL_VERSION}'] = hw_forecast
        else:
            results[f'holt_winters_{MODEL_VERSION}'] = np.full(forecast_steps, y_train.mean())
    except Exception as e:
        results[f'holt_winters_{MODEL_VERSION}'] = np.full(forecast_steps, y_train.mean())

    # 3. SARIMA
    try:
        if len(y_train) >= 21:
            sarima_model = SARIMAX(
                y_train,
                order=(1, 1, 1),
                seasonal_order=(1, 1, 1, 7),
                enforce_stationarity=False,
                enforce_invertibility=False
            ).fit(disp=False)
            sarima_forecast = sarima_model.forecast(steps=forecast_steps)
            results[f'sarima_{MODEL_VERSION}'] = sarima_forecast
        else:
            results[f'sarima_{MODEL_VERSION}'] = np.full(forecast_steps, y_train.mean())
    except Exception as e:
        results[f'sarima_{MODEL_VERSION}'] = np.full(forecast_steps, y_train.mean())

    # 4. Prophet (if available)
    if PROPHET_AVAILABLE:
        try:
            if len(y_train) >= 14:
                prophet_df = pd.DataFrame({
                    'ds': y_train.index,
                    'y': y_train.values
                })

                prophet_model = Prophet(
                    daily_seasonality=False,
                    weekly_seasonality=True,
                    yearly_seasonality=True if len(y_train) >= 365 else False,
                    changepoint_prior_scale=0.05
                )

                prophet_model.fit(prophet_df)

                # Create future dataframe
                future_dates = pd.date_range(
                    start=y_train.index[-1] + pd.Timedelta(days=1),
                    periods=forecast_steps,
                    freq='D'
                )

                future_df = pd.DataFrame({'ds': future_dates})
                prophet_forecast = prophet_model.predict(future_df)['yhat'].values
                results[f'prophet_{MODEL_VERSION}'] = prophet_forecast
            else:
                results[f'prophet_{MODEL_VERSION}'] = np.full(forecast_steps, y_train.mean())
        except Exception as e:
            results[f'prophet_{MODEL_VERSION}'] = np.full(forecast_steps, y_train.mean())

    return results

# %% Hybrid Neural Models (if enabled)
def prepare_neural_data(y_train, X_train, lookback_window=14):
    """Prepare data for neural network models"""
    if len(y_train) < lookback_window + 1:
        return None, None, None, None

    # Create sequences for LSTM/RNN
    X_sequences, y_sequences = [], []

    for i in range(lookback_window, len(y_train)):
        X_sequences.append(y_train.iloc[i-lookback_window:i].values)
        y_sequences.append(y_train.iloc[i])

    X_sequences = np.array(X_sequences)
    y_sequences = np.array(y_sequences)

    return X_sequences, y_sequences, None, lookback_window

def fit_hybrid_neural_models_v1(y_train, X_train, forecast_steps):
    """Hybrid neural models combining classical time series with deep learning"""

    results = {}

    if not ENABLE_NEURAL or not KERAS_AVAILABLE:
        print("⚠️ Neural models disabled")
        return results

    # Prepare data for neural networks
    X_seq, y_seq, market_features, lookback = prepare_neural_data(y_train, X_train, lookback_window=14)

    if X_seq is None or len(X_seq) < 10:
        print("⚠️ Insufficient data for neural models")
        return results

    # Simple LSTM
    try:
        lstm_model = Sequential([
            LSTM(50, return_sequences=False, input_shape=(lookback, 1)),
            Dropout(0.2),
            Dense(25),
            Dense(1)
        ])

        lstm_model.compile(optimizer='adam', loss='mse')

        X_lstm = X_seq.reshape(-1, lookback, 1)
        lstm_model.fit(X_lstm, y_seq, epochs=50, batch_size=8, verbose=0)

        # Generate forecast
        last_sequence = y_train.tail(lookback).values.reshape(1, lookback, 1)
        lstm_forecast_base = lstm_model.predict(last_sequence, verbose=0)[0, 0]

        # Apply trend for multi-step forecast
        if len(y_train) > 1:
            trend = (y_train.iloc[-1] - y_train.iloc[-2])
            lstm_forecast = [lstm_forecast_base + trend * (i + 1) for i in range(forecast_steps)]
        else:
            lstm_forecast = [lstm_forecast_base] * forecast_steps

        results[f'lstm_{MODEL_VERSION}'] = np.array(lstm_forecast)
    except Exception as e:
        results[f'lstm_{MODEL_VERSION}'] = np.full(forecast_steps, y_train.mean())

    return results

# %% Model Evaluation Framework
def calculate_mase(y_true, y_pred, y_train, seasonal_period=7):
    """Calculate Mean Absolute Scaled Error (MASE)"""

    # Calculate MAE of the model
    model_mae = mean_absolute_error(y_true, y_pred)

    # Calculate MAE of seasonal naive benchmark on training data
    if len(y_train) > seasonal_period:
        seasonal_naive_errors = []
        for i in range(seasonal_period, len(y_train)):
            seasonal_naive_pred = y_train.iloc[i - seasonal_period]
            seasonal_naive_errors.append(abs(y_train.iloc[i] - seasonal_naive_pred))

        seasonal_naive_mae = np.mean(seasonal_naive_errors)

        # Avoid division by zero
        if seasonal_naive_mae == 0:
            seasonal_naive_mae = 1e-10

        mase = model_mae / seasonal_naive_mae
    else:
        # Fallback to naive MAE if insufficient data
        naive_mae = np.mean([abs(y_train.iloc[i] - y_train.iloc[i-1])
                           for i in range(1, len(y_train))])
        if naive_mae == 0:
            naive_mae = 1e-10
        mase = model_mae / naive_mae

    return mase

def evaluate_model_v1(y_true, y_pred, y_train, model_name):
    """Comprehensive model evaluation with all metrics including MASE"""

    # Remove any NaN values
    mask = ~(np.isnan(y_true) | np.isnan(y_pred))
    y_true_clean = y_true[mask]
    y_pred_clean = y_pred[mask]

    if len(y_true_clean) == 0:
        return {
            'model': model_name,
            'mae': np.nan,
            'rmse': np.nan,
            'mape': np.nan,
            'mase': np.nan,
            'n_obs': 0
        }

    # Calculate all metrics
    mae = mean_absolute_error(y_true_clean, y_pred_clean)
    rmse = np.sqrt(mean_squared_error(y_true_clean, y_pred_clean))
    mape = mean_absolute_percentage_error(y_true_clean, y_pred_clean) * 100
    mase = calculate_mase(y_true_clean, y_pred_clean, y_train)

    return {
        'model': model_name,
        'mae': mae,
        'rmse': rmse,
        'mape': mape,
        'mase': mase,
        'n_obs': len(y_true_clean)
    }

# %% Comprehensive Evaluation
def run_comprehensive_evaluation_v1():
    """Run all V1 models on all CV splits with proper evaluation framework"""

    print("🎯 RUNNING COMPREHENSIVE V1 MODEL EVALUATION")
    print("=" * 50)

    all_results = []

    for split_idx, split in enumerate(cv_splits):
        print(f"\n📊 Evaluating Split {split_idx + 1}/{len(cv_splits)}")

        # Get raw train/test data
        train_data_raw = df_raw.loc[split['train_idx']]
        test_data_raw = df_raw.loc[split['test_idx']]

        # Apply regime-appropriate training window
        if len(train_data_raw) > 90:
            train_data_raw = train_data_raw.tail(90)
        elif len(train_data_raw) > 60:
            train_data_raw = train_data_raw.tail(60)

        # Apply feature engineering per split
        train_features, test_features = create_features_v1(train_data_raw, test_data_raw)

        y_train = train_features['calls']
        y_test = test_data_raw['calls'].values
        forecast_steps = len(test_data_raw)

        # Prepare ML features
        feature_cols = [col for col in train_features.columns
                       if col not in ['calls'] and not col.startswith('calls_lag')]
        X_train_ml = train_features[feature_cols].dropna()

        # 1. BASIC STATISTICAL MODELS V1
        basic_results = fit_all_basic_models_v1(y_train, forecast_steps)

        for model_name, pred in basic_results.items():
            if len(pred) == len(y_test):
                metrics = evaluate_model_v1(y_test, pred, y_train, model_name)
                metrics['split'] = split_idx + 1
                all_results.append(metrics)

        # 2. ADVANCED TIME SERIES MODELS V1
        advanced_results = fit_advanced_time_series_v1(y_train, forecast_steps)

        for model_name, pred in advanced_results.items():
            if len(pred) == len(y_test):
                metrics = evaluate_model_v1(y_test, pred, y_train, model_name)
                metrics['split'] = split_idx + 1
                all_results.append(metrics)

        # 3. HYBRID NEURAL MODELS V1 (if enabled)
        if ENABLE_NEURAL and KERAS_AVAILABLE:
            neural_results = fit_hybrid_neural_models_v1(y_train, X_train_ml, forecast_steps)

            for model_name, pred in neural_results.items():
                if len(pred) == len(y_test):
                    metrics = evaluate_model_v1(y_test, pred, y_train, model_name)
                    metrics['split'] = split_idx + 1
                    all_results.append(metrics)

    # Convert to DataFrame and calculate averages
    results_df = pd.DataFrame(all_results)

    if len(results_df) == 0:
        print("❌ No results generated!")
        return None, None

    # Calculate average performance across splits
    avg_results = results_df.groupby('model').agg({
        'mae': 'mean',
        'rmse': 'mean',
        'mape': 'mean',
        'mase': 'mean',
        'n_obs': 'sum'
    }).round(2)

    # Sort by MASE (primary ranking metric)
    avg_results = avg_results.sort_values('mase')

    print(f"\n✅ V1 Model Evaluation Complete!")
    print(f"📊 {len(avg_results)} models evaluated across {len(cv_splits)} splits")

    return results_df, avg_results

# %% Performance Summary
def create_performance_summary_v1(avg_results):
    """Create performance summary in requested format"""

    if avg_results is None or len(avg_results) == 0:
        print("❌ No results available for performance summary")
        return None

    print("📊 MODEL PERFORMANCE SUMMARY V1")
    print("=" * 50)

    # Create summary table
    summary = avg_results[['mae', 'rmse', 'mape', 'mase']].copy()

    # Ensure seasonal_naive shows exactly 1.00 MASE
    seasonal_naive_models = [idx for idx in summary.index if 'seasonal_naive' in idx.lower()]
    for model in seasonal_naive_models:
        if model in summary.index:
            summary.loc[model, 'mase'] = 1.00

    # Sort by MASE for final ranking
    summary = summary.sort_values('mase')

    # Format the display
    print("Model Performance Summary:")
    print(f"{'Model':<25} {'MAE':<10} {'RMSE':<10} {'MAPE':<8} {'MASE':<8}")
    print("-" * 65)

    for model_name, row in summary.iterrows():
        display_name = model_name.replace('_V1', '').replace('_', ' ').title()
        if len(display_name) > 24:
            display_name = display_name[:21] + "..."

        print(f"{display_name:<25} {row['mae']:<10.2f} {row['rmse']:<10.2f} {row['mape']:<8.2f} {row['mase']:<8.2f}")

    # Performance analysis
    print(f"\n🏆 PERFORMANCE ANALYSIS V1")
    print("-" * 30)

    best_model = summary.index[0]
    best_mase = summary.iloc[0]['mase']

    print(f"🥇 Best Model: {best_model.replace('_V1', '').replace('_', ' ').title()}")
    print(f"   MASE: {best_mase:.2f}")

    if best_mase < 0.8:
        print("   ✅ EXCELLENT: Significantly better than seasonal naive")
    elif best_mase < 1.0:
        print("   ✅ GOOD: Better than seasonal naive benchmark")
    elif best_mase < 1.2:
        print("   ⚠️ CLOSE: Nearly as good as seasonal naive")
    else:
        print("   ❌ POOR: Worse than seasonal naive benchmark")

    return summary

# %% Main Execution
print("🚀 Starting V1 Model Evaluation...")
results_df_v1, avg_results_v1 = run_comprehensive_evaluation_v1()

if avg_results_v1 is not None:
    # Create and display performance summary
    summary_v1 = create_performance_summary_v1(avg_results_v1)

    print(f"\n✅ Phase 1 (V1 Models) Complete!")
    print(f"🎯 Ready for Phase 2: Residual Treatment (V2 Models)")
else:
    print(f"\n❌ Phase 1 evaluation failed")

# %% Final Status
print(f"\n" + "="*60)
print("🎉 PHASE 1 (V1 MODELS) COMPLETE!")
print("="*60)
print("📊 Next Steps:")
print("   1. Review V1 performance summary above")
print("   2. Prepare Phase 2 notebook (Residual Treatment)")
print("   3. Focus on top-performing V1 models for V2 enhancement")
print("="*60)

🖥️ COMPUTATIONAL ENVIRONMENT CHECK
✅ GPU Available:
Fri Sep 19 16:27:17 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.54.15              Driver Version: 550.54.15      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  Tesla T4                       Off |   00000000:00:04.0 Off |                    0 |
| N/A   38C    P8              9W /   70W |       0MiB /  15360MiB |      0%      Default |
|                                         |                        |                  N/

/usr/local/lib/python3.12/dist-packages/statsmodels/tsa/base/tsa_model.py:473: ValueWarning: No frequency information was provided, so inferred frequency D will be used.
  self._init_dates(dates, freq)
/usr/local/lib/python3.12/dist-packages/statsmodels/tsa/base/tsa_model.py:473: ValueWarning: No frequency information was provided, so inferred frequency D will be used.
  self._init_dates(dates, freq)
/usr/local/lib/python3.12/dist-packages/statsmodels/tsa/base/tsa_model.py:473: ValueWarning: No frequency information was provided, so inferred frequency D will be used.
  self._init_dates(dates, freq)
/usr/local/lib/python3.12/dist-packages/statsmodels/tsa/base/tsa_model.py:473: ValueWarning: No frequency information was provided, so inferred frequency D will be used.
  self._init_dates(dates, freq)
DEBUG:cmdstanpy:input tempfile: /tmp/tmpgasry_z2/tkl9qc5y.json
DEBUG:cmdstanpy:input tempfile: /tmp/tmpgasry_z2/n6_t4rg1.json
DEBUG:cmdstanpy:idx 0
DEBUG:cmdstanpy:running CmdStan, num_threads


📊 Evaluating Split 2/5
🛠️ FEATURE ENGINEERING V1 (Leakage-Free)
✅ Using ONLY training data statistics
✅ Created 35 features for training data
✅ Applied same transformations to test data
📈 FITTING ADVANCED TIME SERIES MODELS V1


/usr/local/lib/python3.12/dist-packages/statsmodels/tsa/base/tsa_model.py:473: ValueWarning: No frequency information was provided, so inferred frequency D will be used.
  self._init_dates(dates, freq)
/usr/local/lib/python3.12/dist-packages/statsmodels/tsa/base/tsa_model.py:473: ValueWarning: No frequency information was provided, so inferred frequency D will be used.
  self._init_dates(dates, freq)
/usr/local/lib/python3.12/dist-packages/statsmodels/tsa/base/tsa_model.py:473: ValueWarning: No frequency information was provided, so inferred frequency D will be used.
  self._init_dates(dates, freq)
/usr/local/lib/python3.12/dist-packages/statsmodels/tsa/base/tsa_model.py:473: ValueWarning: No frequency information was provided, so inferred frequency D will be used.
  self._init_dates(dates, freq)
DEBUG:cmdstanpy:input tempfile: /tmp/tmpgasry_z2/nx38jgyc.json
DEBUG:cmdstanpy:input tempfile: /tmp/tmpgasry_z2/kwi_cv3o.json
DEBUG:cmdstanpy:idx 0
DEBUG:cmdstanpy:running CmdStan, num_threads


📊 Evaluating Split 3/5
🛠️ FEATURE ENGINEERING V1 (Leakage-Free)
✅ Using ONLY training data statistics
✅ Created 35 features for training data
✅ Applied same transformations to test data
📈 FITTING ADVANCED TIME SERIES MODELS V1


...


📊 Evaluating Split 4/5
🛠️ FEATURE ENGINEERING V1 (Leakage-Free)
✅ Using ONLY training data statistics
✅ Created 35 features for training data
✅ Applied same transformations to test data
📈 FITTING ADVANCED TIME SERIES MODELS V1


/usr/local/lib/python3.12/dist-packages/statsmodels/tsa/base/tsa_model.py:473: ValueWarning: No frequency information was provided, so inferred frequency D will be used.
  self._init_dates(dates, freq)
/usr/local/lib/python3.12/dist-packages/statsmodels/tsa/base/tsa_model.py:473: ValueWarning: No frequency information was provided, so inferred frequency D will be used.
  self._init_dates(dates, freq)
/usr/local/lib/python3.12/dist-packages/statsmodels/tsa/base/tsa_model.py:473: ValueWarning: No frequency information was provided, so inferred frequency D will be used.
  self._init_dates(dates, freq)
/usr/local/lib/python3.12/dist-packages/statsmodels/tsa/base/tsa_model.py:473: ValueWarning: No frequency information was provided, so inferred frequency D will be used.
  self._init_dates(dates, freq)
DEBUG:cmdstanpy:input tempfile: /tmp/tmpgasry_z2/mg7pfdr8.json
DEBUG:cmdstanpy:input tempfile: /tmp/tmpgasry_z2/v3bwb4sb.json
DEBUG:cmdstanpy:idx 0
DEBUG:cmdstanpy:running CmdStan, num_threads


📊 Evaluating Split 5/5
🛠️ FEATURE ENGINEERING V1 (Leakage-Free)
✅ Using ONLY training data statistics
✅ Created 35 features for training data
✅ Applied same transformations to test data
📈 FITTING ADVANCED TIME SERIES MODELS V1


/usr/local/lib/python3.12/dist-packages/statsmodels/tsa/base/tsa_model.py:473: ValueWarning: No frequency information was provided, so inferred frequency D will be used.
  self._init_dates(dates, freq)
/usr/local/lib/python3.12/dist-packages/statsmodels/tsa/base/tsa_model.py:473: ValueWarning: No frequency information was provided, so inferred frequency D will be used.
  self._init_dates(dates, freq)
/usr/local/lib/python3.12/dist-packages/statsmodels/tsa/base/tsa_model.py:473: ValueWarning: No frequency information was provided, so inferred frequency D will be used.
  self._init_dates(dates, freq)
/usr/local/lib/python3.12/dist-packages/statsmodels/tsa/base/tsa_model.py:473: ValueWarning: No frequency information was provided, so inferred frequency D will be used.
  self._init_dates(dates, freq)
DEBUG:cmdstanpy:input tempfile: /tmp/tmpgasry_z2/a3ilo2w5.json
DEBUG:cmdstanpy:input tempfile: /tmp/tmpgasry_z2/4ghnghio.json
DEBUG:cmdstanpy:idx 0
DEBUG:cmdstanpy:running CmdStan, num_threads


✅ V1 Model Evaluation Complete!
📊 10 models evaluated across 5 splits
📊 MODEL PERFORMANCE SUMMARY V1
Model Performance Summary:
Model                     MAE        RMSE       MAPE     MASE    
-----------------------------------------------------------------
Holt Winters              594.81     738.23     7.77     0.73    
Sarima                    640.33     783.32     8.43     0.79    
Ets                       659.67     816.86     8.35     0.81    
Seasonal Naive            640.14     849.43     7.86     1.00    
Prophet                   972.63     1104.41    12.76    1.19    
Median                    1397.86    1830.64    22.56    1.71    
Mean                      1483.98    1766.83    22.60    1.81    
Naive                     1753.29    2303.62    28.85    2.14    
Drift                     1790.73    2333.55    29.33    2.19    
Lstm                      7673.40    7956.11    97.10    9.44    

🏆 PERFORMANCE ANALYSIS V1
------------------------------
🥇 Best Model: Holt Wi